# Encuesta de Hospitales Exploration

## Import Libraries

In [ ]:
import pandas as pd
import csv

## Read Data

In [ ]:

# _enHospObj = pd.ExcelFile('dataset/Encuesta Nacional de Hospitales versión Diálisis(Responses).xlsx')
# enHospDf = pd.read_excel('dataset/Encuesta Nacional de Hospitales versión Diálisis(Responses).xlsx',sheet_name=_enHospObj.sheet_names[0])


enHospDf = pd.read_csv('dataset/Encuesta Nacional de Hospitales.csv')

## First Exploration iteration

In [ ]:
# 118 Columns

# enHospDf.info() # float(10 columns), int(64 columns), object(93 columns)

enHospDf.isna().sum().sum() #17,539 null datapoints

nullValues = enHospDf.isnull().mean().sort_values(ascending = False) # Percentage of Null Values per column

nullValues[nullValues > 0.1].count() # 46 columns have more than 10% null Values

nullValues.head() # Top 5 Variables have more than 87% of their values missing


### Group Variables by heuristics
 The dataset is divided in 11 different sections:
 

1. ID Data
2. General operability Var 5 to var 8
3. Specific operability <br> - In this section the 3 questions are likert scales. Thus, the high number of columns 
4. Dialysis Service
5. Nutrition Service
6. Attention Quality
7. Personnel Availability
8. Public Service
9. Mortality Impact
10. Violence and Protests
11. Epidemiological Emergency

In [ ]:
# Section 0
idData = enHospDf.iloc[:,0:6]

# Section 1 General Operability 
genOp = enHospDf.iloc[:,6:10]

# Section 2 Especific Operability
espOp = enHospDf.iloc[:,10:47]

# Section 3 Dialysis Service
dialServ = enHospDf.iloc[:,47:75]

# Section 4 Nutrition Service
nutriServ = enHospDf.iloc[:,75:83]

# Section 5 Attention Quality
attQual = enHospDf.iloc[:,83:85]

# Section 6 Personnel Availability
personelDisp = enHospDf.iloc[:,85:95]

# Section 7 Public Services
publicServ = enHospDf.iloc[:,95:107]

# Section 8 Mortality Impact
mortImpact = enHospDf.iloc[:,107:109]

# Section 9 Violence and protests
violence = enHospDf.iloc[:,109:117]

# Section 10 Epidemy Emergency
epiEmerg = enHospDf.iloc[:,-1]
# enHospDf.columns[-4:-1]

#### General Operability 

##### This section contains: 
1. <b>Number of architectural beds</b>: Maximum number of beds that the hospital was designed for. 
<br>
2. <b>Number of operative beds</b>: Number of beds that are currently functional.
<br>
3. <b>Number of operative emergency beds</b>: Number of functional beds in ER.
<br>
4. <b>Number of operative hospital pavilions</b>:
 - Pavilion Definition: One of a group of related buildings (cambridge dictionary)
 <br><br>

##### Possible Business Questions: <br>
This section portrays the most elemental hospital operativity.

- What are the current conditions of the hospital in terms of its operative beds and pavilions?
- How the original number of beds differ from the actual number of beds that are operative?
- Which hospitals have the highest amount of operative beds and Pavilions? 
- Which states have the highest/lowest amount of operative beds and Pavilions? 
<br><br> 

##### Questions to Client

- The the sum of operative beds and operative emergency beds is higher the number of architectural beds. If the number of architectural beds is the maximum amount of beds that the hospital was designed for why the sum of all the beds is higher? 


<br><br> 

Source: http://maiquiflores.over-blog.es/article-definiciones-basicas-de-los-indicadores-de-la-gestion-hospitalaria-en-venezuela-2014-123693467.html

Better source pending

In [ ]:
# genOp.head()

# Row 615 has corrupt datapoint. -> 8|8
# Check with client how the quality of data that is submitted.
_corruptGenOp = genOp[genOp['5.- Número de camas arquitectónicas'].apply(lambda x: type(x) == str)] 

genOp = genOp.drop(_corruptGenOp.index).astype(int)


In [ ]:
# 131 rows where the sum of operative beds and operative emergency beds is higher to number of arquitectural beds

genOp[(genOp.iloc[:,1] + genOp.iloc[:,2]) > genOp.iloc[:,0]]

#### Specific Operability 
No null values
<br> 

##### Possible Questions
- What is the equipment that lacks the most?
- Which hospitals have the most scarce equipment?
- Which hospitals have the most complete equipment?
- What are the medicines that lack the most?
- What are the medicines that hospitals have the most availability?
- Which hospitals have most scarcity of medicines?
- Which hospitals have the most amount of medicines? 
- What is the availability of medicines per hospital? (interactive viz)
- What is the availability of medicines per state? 
    - <i>In order to track demand of medicines as well</i>
<br>


##### Variables

###### 9.- Equipment <br>
- UTI: The UTI is the Intermediate Treatment Unit that has specialized personnel and equipment that allows patients who, after the critical stage, require constant surveillance, attention and observation, prior to transfer to a conventional room.
- UTI(P): Definition Pending
- Emergency: How much time has the ER been operative.
- Pavilion: How much time have the Pavilions been operative.
- Laboratory: How much time has the Laboratory been operative. 
- Ecography: How much time has the ecography been operative. 
- TAC/TMN: Axial Computarized Tomography [TAC], TMN To be defined. 
- X Rays: How much time has the X-Ray equipment has been operative. 


###### 10.- Medicines <br> 
- Adrenalin:
- Atropine:
- Dopamine:
- Cephalosporins / beta-lactams:
- Aminoglycosides / quinolones:
- Vancomycin / Clindamycin:
- Local anesthesia (lidocaine:
- Minor analgesic (NSAIDs, dipyrone, etc.:
- Major analgesic (morphine, demerol:
- Fluidotherapics (0.9%, Gluc 5%,) / infusion equipment:
- Diazepam / DPH:
- Heparin:
- Steroids:
- Insulin:
- Med inhaled for asthma:
- Antihypertensive:
- Defibrillator:
- TOT / intubation:
- Yelco / central roads:
- Oxygen / suction:
- Minor analgesics:
- Major analgesics:
- Anesthetic gases:
- EV anesthetics:
- Relaxing:
- TOT / intubation equipment:
- Patient lingerie kit:
- Disposable health personnel (cap, gloves, gown:
- Oxygen / suction:


In [ ]:
# from PyDictionary import PyDictionary

# dictionary = PyDictionary() # In case we need to look up for the medicine definitions. 

# dictionary.meaning('Cephalosporins')
espOp.head()

#### Dialysis Service 
<br> 
<i> Note: All hospitals that don't have a dialysis service will not have datapoints in the rest of the section's questions.178 rows don't have dialysis service<i>  
<br>

##### Overall Dialysis Service
- How many hospitals count with a dialysis service?
- How many hospitals don't count with a dialysis service? 

<br>

##### Non-Operative Dialysis Service
- How many hospitals have a dialysis service that is not operative?
- Why these hospitals dialysis service is not operative? 
- What is the name/location of these hospitals that don't have operative dialysis.  

<br>

##### Operative Dialysis Service
<i> Note: Peritoneal and Hemodialysis are focused on the hospitals that provide them rather than the number of patients that each hospital attend. The reasoning behind this is to clearly communicate the overall state of dialysis services in Venezuela.   
<br> 
- Overall Dialysis Service:
    - How many and which hospitals have an operative dialysis service
    - What is the number of weekly dialysis patients that each hospital has attended across time? 
        - <i> This question is designed to inspect the patient service trend across time <i>
    - What is the total average number of hemodialysis and peritoneal dialysis patients by hospital?
- Peritoneal Dialysis
    - What is the number of hospitals that have peritoneal dialysis out of the total?
    - What are the reason why hospitals don't provide peritoneal dialysis?
    - Which hospitals don't provide peritoneal dialysis?
- Hemodialysis
    - What is the number of hospitals that have peritoneal dialysis out of the total? 
    - What is the overall percentage of acute and chronic hemodialysis patients? 

<br> 
##### Dialysis Materials Availability

- Questions: 
    - What are the materials that lack the most?
    - What are the materials that hospitals have the most availability?
    - Which hospitals have most scarcity of materials?
    - Which hospitals have the most amount of materials? 
    - What is the availability of materials per hospital? (interactive viz)
    - What is the availability of materials per state? 
        - <i>In order to track demand of medicines as well</i>
    - How many and which hospitals have an operative inverse osmosis plant?
<br>


###### Type of Materials
- Filters
- Connection lines
- Hemodialysis kit
- Iron
- Complex B
- Calcium
- Zemblar®

<br>

##### Dialysis Personnel Availability
- Questions: 
    - What personnel lacks the most? 
    - What personnel has the most availability?
    - Which hospitals have the most lack & availability of dialysis personal?
    - Which states have the most lack & availability of dialysis personal?
    - What is the average number of days that each type of personnel is missing by hospital and by state?



###### Type of DIalysis Personnel
- Nephrologist Specialist
- General practitioner
- Resident
- Graduate nurse
- Nurse specialist in nephrology


In [ ]:
dialServ

# Hospitals that never had dialysis service
# dialServ[dialServ['12.- ¿Existe servicio de diálisis en su hospital?'] == 'No'].isnull().sum()

# Hospitals that have a dialysis service
# dialServ[dialServ['12.- ¿Existe servicio de diálisis en su hospital?'] == 'Sí']

# Hospitals that have dialysis service that is not operative
# dialServ[(dialServ['12.- ¿Existe servicio de diálisis en su hospital?']=='Sí') & (dialServ['13.-¿Está operativo el servicio de diálisis?'] == 'No')]


#### Nutrition Service
- Questions:
    - Overall Nutrition Service
        - How many hospitals out of the total count with nutrition service?
        - Which hospitals don't count with nutrition service in the last week?
        - From the hospitals that count with nutrion service how many don't have an operative nutrion service?
        - From these hospitals, what is the most frequent reason that the nutrition service is not operative? 
        - Which hospitals have a nutrition service that is not operative in the last week?  
        - What is the average operability of the nutrition service in terms of time?
    - Meals Frequency: 
        - How frequent are the daily meals overall in terms of time?
        - Which hospitals don't count with a nutrition service in the last week? 
        - Which hospitals have a low frequency of daily meals?
    - Meals quality: 
        - What is the quality of nutrition service across time? 
        - What is the quality of nutrition service of all hospitals in the last week? 
        - Which have the lowest quality of nutrition service in the last week? 
    - Baby Milk Formulas: 
        - What is the overall frequency of the supply of milk formulas for babies? 
        - Which hospitals have the lowest/highest frequency of baby formulas in the past week? 

In [ ]:
# nutriServ['27.- ¿Existe servicio de nutrición en su hospital?'].value_counts()
# nutriServ[nutriServ['27.- ¿Existe servicio de nutrición en su hospital?'] == 'No'].isnull().sum()

# nutriServ[(nutriServ['27.- ¿Existe servicio de nutrición en su hospital?'] == "Sí") & (nutriServ['28.-¿Está operativo el servicio de nutrición?'] == 'No')]
nutriServ

#### Attention Quality

This section is comprised by the interval of time between arriving to emergency and being attended. 

- Pneumonia Patients: 
    - What is the overall average wait time for pneumonia patients to be treated in ER? 
    - What is the average wait time for pneumonia patients per location (state) to be treated? 
    - Which hospitals have the fastest attention of pneumonia patients in the last week? 
    - Which hospitals have the slowest attention of pneumonia patients in the last week? 
<br><br>
- Acute myocardial infarction patients (heart attack): 
    - What is the overall average wait time for heart attack patients to be treated in ER? 
    - What is the average wait time for heart attack patients per location (state) to be treated? 
    - Which hospitals have the fastest attention of heart attack patients in the last week? 
    - Which hospitals have the slowest attention of heart attack patients in the last week? 

In [ ]:
attQual

#### Personnel Availability

This section consists of the number of health professionals that were available per hospital in the past week. 

##### Questions
- Which types of the health professionals are overall most available?
- Which types of health professionals are overall less available?
- How overall day and night shift professional availability vary?
- Which hospitals have less overall availability of health professionals?
    - Which are these health professionals? 
    - Where are these hospitals located? 
- Which hospitals have the most/least availability of health professionals in the last week?

In [ ]:
personelDisp.head()

#### Public Service

- In which areas water service lacked the most?
- Which hospitals lack water service in the last week?
- What is the overall percentage of power outages?
- What is the average power outages per hospital?
- Which hospitals had the highest number of power outages in the last week? 
- What is the average number of power outages per hospital / per state?
    - From these power outages,which equipment equipment has broken?
- Which hospitals/states are in most need of power plants?
    - Most malfunctioning power plants
    - At least 1 death due to power outages.
- In which hospitals patients have died due to power outages? 
    - How many?

In [ ]:
publicServ

# publicServ['51.- Duración promedio de fallos por semana del servicio luz eléctrica '].value_counts()
# publicServ['53.- ¿Cuáles equipos fallaron posterior a fallas de energía la semana pasada? '].value_counts()[publicServ['53.- ¿Cuáles equipos fallaron posterior a fallas de energía la semana pasada? '].value_counts() <= 10]

#### Mortality Impact


- How many cardiovascular deaths per state/hospitals happened due to institutional failures?
    - From these, which are the most common institutional failures?
- How many trauma deaths per state/hospitals happened due to institutional failures?
    - From these, which are the most common institutional failures?
- In the last week how many deaths due to institutional failures occured?
    - From which states
    - From which hospitals
    - Cardiovascular vs trauma


<br>
<br>

______________________
Preguntar:
Cual es el tipo de falla institucional que se registra? Se puede utilizar la definición como definición?

Falla Institucional definicion: 

Las causas más comunes de conflictos institucionales son fallas de comunicación, deficiencia en la estructura institucional y problemas en el comportamiento individual en las instituciones. Además, los recursos limitados son también considerados causa de conflicto, cuando involucran la inadecuada estructura física, la falta de recursos financieros, la disponibilidad de aparados estropeados y obsoletos, generando conflictos operacionales que afectan el desarrollo de los profesionales. http://scielo.isciii.es/scielo.php?script=sci_arttext&pid=S1132-12962012000100013

In [ ]:
mortImpact[(mortImpact['57.- Número de muertes cardiovasculares interpretada como falla institucional (emergencia )'] >= 2.0)]

#### Violence and Protests

- How many protests have occurred per month?
- Where these protests occurred?
- From these protests, how many were done by medical personnel, nurses, other health professionals, and others?
- How many protests have occurred per state?
- Which are the most common violence against protesters overall and per state?

In [ ]:
violence

#### Epidemiology

- What is the number of deaths per disease across time?
- Where these deaths occurred?
- What is the frequency/type/location of each disease in the last week?

In [ ]:
epiEmerg.value_counts()

### First Translation
This is a quick translation of the columns using google translate's API

In [ ]:
from googletrans import Translator
translator = Translator()
translation = translator.translate(enHospDf.columns.to_list(), src= 'spanish', dest='english')


In [ ]:
# Append translated columns to list
transCols = []

for i in range(enHospDf.shape[1]): 
    transCols.append(translation[i].text)
    
# create csv with translated columns
# with open ('dataset/translatedCols_1.csv','w') as myfile:
#     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#     wr.writerow(transCols)

In [ ]:
# Create new DF with translated columns
enHospDfTrans = enHospDf.copy() # Copy original DF
enHospDfTrans.columns = transCols # Overwrite Column Names

# Write Csv
# enHospDfTrans.to_csv('dataset/NationalSurveyHospitals_Translated.csv',index = False)

enHospDfTrans.head()